In [9]:
!rm -rf Pytorch_Modules

In [9]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import DataLoader

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the Pytorch_Modules directory, download it from GitHub if it doesn't work
try:
    from Pytorch_Modules import engine, prediction, utils, alexnet
except:
    # Get the Pytorch_Modules scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/yasinahmadpoor/FromScratch
    !mv FromScratch/Pytorch_Modules .
    !rm -rf FromScratch
    from Pytorch_Modules import engine, prediction, utils


device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
def get_train_test_loader(data_dir,
                           batch_size,
                           augment,
                           shuffle=True):

    # define normalizer
    normalize = transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])    # Normalize a tensor image with mean and standard deviation for n channels, this transform will normalize each channel of the input.


    # define transforms
    valid_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),          # transforms.ToTensor() automatically scales the images to [0,1] range
                normalize])


    if augment:
            train_transform = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
    else:
            train_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),
                normalize,
            ])

    # download and create datasets
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, transform=train_transform, download=True)
    valid_dataset = datasets.CIFAR10(root=data_dir, train=False, transform=valid_transform, download=True)

    # define the data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = shuffle)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_dataset, valid_dataset, train_loader, valid_loader)

BATCH_SIZE = 64
train_data, test_data, train_dataloader, test_dataloader = get_train_test_loader(data_dir = 'CIFAR10_data',
                                                    batch_size = BATCH_SIZE,
                                                    augment = False)
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Files already downloaded and verified
Files already downloaded and verified
Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f19e40448b0>, <torch.utils.data.dataloader.DataLoader object at 0x7f19e177b700>)
Length of train dataloader: 782 batches of 64
Length of test dataloader: 157 batches of 64


In [18]:
# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model = alexnet.AlexNet(input_channels=3,
                        num_classes=10).to(device)

In [16]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
# model.eval()

# # Set the manual seeds
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# NUM_CLASSES = 10

# # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# for param in model.parameters():
#   param.requires_grad = False


# num_ftrs = model.classifier[6].in_features
# model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [19]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 227, 227), # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [32, 3, 227, 227]    [32, 10]             --                   True
├─Sequential (layer1)                    [32, 3, 227, 227]    [32, 96, 27, 27]     --                   True
│    └─Conv2d (0)                        [32, 3, 227, 227]    [32, 96, 55, 55]     34,944               True
│    └─BatchNorm2d (1)                   [32, 96, 55, 55]     [32, 96, 55, 55]     192                  True
│    └─ReLU (2)                          [32, 96, 55, 55]     [32, 96, 55, 55]     --                   --
│    └─MaxPool2d (3)                     [32, 96, 55, 55]     [32, 96, 27, 27]     --                   --
├─Sequential (layer2)                    [32, 96, 27, 27]     [32, 256, 13, 13]    --                   True
│    └─Conv2d (0)                        [32, 96, 27, 27]     [32, 256, 27, 27]    614,656              True
│    └─BatchNorm2d

In [20]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [21]:
# Start training with help from engine.py
NUM_EPOCHS = 5
result = engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

11:59:06 --- Epoch: 1	Train loss: 2.0093	Test loss: 1.7117	Train accuracy: 25.55	Test accuracy: 0.32


0it [00:00, ?it/s]

0it [00:00, ?it/s]

12:02:09 --- Epoch: 2	Train loss: 1.6612	Test loss: 1.4853	Train accuracy: 36.94	Test accuracy: 0.36


0it [00:00, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")